# Landbedeckungsklassifikation anhand von Luftbildern

CAS Machine Intelligence, Modul Machine Learning, Timo Grossenbacher, 8. April 2019

## Einleitung & Datensatz

Die automatische Klassifizierung von Luft- und Satellitenbildern hat unterschiedlichste Anwendungen in der Forschung, Privatwirtschaft und bei der öffentlichen Hand. Zum Beispiel kann so das Ausmass der Zersiedelung in einem bestimmten Gebiet oder die Abholzung in einem anderen, automatisch quantifiziert werden. 

Als Übungsanlage für das Modul "Machine Learning" habe ich auf Kaggle einen Datensatz entdeckt, anhand dessen sich eine Landbedeckungsklassifikation mit modernen Methoden des Machine Learning auf Bildebene (im Gegensatz zur Pixelebene) ausprobieren lässt:

Das [SAT-6 Airborne Dataset](https://www.kaggle.com/crawford/deepsat-sat6) umfasst **405'000** Luftbilder einer Grösse von **28x28 Pixel** und mit einem Meter Bodenauflösung. Die Bilder kommen jeweils mit **vier verschiedenen Spektralkanälen: Rot, Grün, Blau und Nahinfrarot**. Die Bilder wurden allesamt gelabelt und jeweils **einer von sechs Landbedeckungsklassen** zugeordnet:

1. barren land ("unfruchtbares/unproduktives Land"), 
2. trees ("Bäume"), 
3. grassland ("Grasland/Grünland"), 
4. roads ("Strassen"), 
5. buildings ("Gebäude"),
6. water bodies ("Gewässer")

Folgend ein paar Beispielbilder:

![Beispielbilder](img/sample_images.png)


### Zielsetzung

Die Zielsetzung der Aufgabe besteht darin, anhand eines **reduzierten Trainingsdatensatzes** (**10 Prozent** der zur Verfügung gestellten 324'000 Trainingsbilder – minus 20 Prozent, die für ein Validationsdatensatz verwendet werden), eine möglichst hohe Genauigkeit ("accuracy") im *vollständigen* Testdatensatz (81'000 Testbilder) zu erreichen. Dabei sollen alle Klassen ausreichend gut erkannt werden, was sich mit dem klassenweisen F1-Wert beurteilen lässt. Dies insbesondere, da die Klassen sehr ungleich verteilt sind. Zum Beispiel gibt es sehr viele Bilder von Gewässern, aber relativ wenig von Strassen. 

Der Trainingsdatensatz wurde bewusst reduziert, um a) die Aufgabe etwas schwieriger zu machen und b) aus Performance-Gründen, denn die Modelle wurden auf einem 4-Kern-Laptop mit 16GB Speicher gerechnet.

### Umgebung & Code

Gearbeitet wurde im ganzen Projekt mit Python 3.6 und den Packages `sklearn`, `pandas` und `numpy` sowie `matplotlib`. Der Code ist unter [https://github.com/grssnbchr/ml_assignment_2019](https://github.com/grssnbchr/ml_assignment_2019) verfügbar.

## Baseline / naiver Ansatz

Ein "naiver" Ansatz in der Bildklassifizierung ist es, alle Spektralkanäle "flach zu drücken" und aneinander zu hängen, woraus ein eher langer Feature-Vektor wird. Im Fall von vier Kanälen und 28x28 Pixel ergibt dies einen Feature-Vektor der Länge 3136. Ich habe diesen Feature-Vektor in ein **Random-Forest-Modell mit 100 Decision Trees** gespiesen.

Zu meinem Erstaunen erreichte bereits diese simple Klassifizierung eine Accuracy von über **96 Prozent** auf dem Validationsdatensatz (20 Prozent des Trainingsdatensatzes). 
Auch die Klassenverteilung war relativ gut balanciert, wie die nachfolgende Confusion Matrix sowie die klassenweisen Performance-Werte zeigen:


![Confusion Matrix](img/confusion_matrix_baseline.png)

Die Matrix ganz links zeigt die Anzahl Klassifizierungen in absoluten Werten (Zeilen: echte Werte, Spalten: vorausgesagte Werte). Die in der Mitte das gleiche, aber relativ (nach Zeilensummen). Die ganz rechts zeigt ebenfalls die relativen Werte, aber mit Nullen in der Diagonalen, damit man die Abweichungen besser sieht. Das grösste Problem zeigt sich bei der Klasse 4 ("Strassen" in diesem Fall), die fälschlicherweise relativ "oft" als Klasse 0 ("Gebäude" in diesem Fall) misklassifiziert wird (= Präzision 91 Prozent). Auch der Recall ist bei den Strassen mit 79 Prozent eher tief, da nur 117 von 149 als "Strassen" klassifizierte Bilder wirklich Strassen sind.

Auch die vormalige Konvertierung der RGB-Kanäle zu Grautönen, wobei lediglich ein Feature-Vektor mit 784 Pixeln (1 Kanal x 28 x 28 Pixel) übrig bleibt, erreichte noch eine Accuracy von rund **84 Prozent**. 

Der **Nachteil** dieses naiven Ansatzes ist die relativ ressourcenintensive Trainingsphase. Auf meiner Maschine dauerte das Trainieren eines Random Forest mit 100 Bäumen und 10 Prozent der Trainingsdaten über eine Minute. Das mag nicht nach viel tönen, aber eine wesentlich optimierte Version des Modells konnte mit weniger als 10 Sekunden trainiert werden (siehe unten).



## Explorative Analyse und Feature Engineering

### Werte standardisieren und Statistiken extrahieren

Unterschiedliche Landbedeckung hat unterschiedliche Spektralsignaturen. Offensichtlicherweise hat zum Beispiel Wasser eher tiefe Werte in allen Kanälen (ausser dem Blaukanal), während (grüne) Vegetation hohe Werte im Nahinfrarot-Bereich aufweist. Da ein SAT-6-Bild lediglich 28x28m Bodenauflösung aufweist, ist die Verteilung dieser Werte über das Bild relativ homogen, und eine "Zusammenfassung" der Information in weniger Dimensionen bietet sich an. Als einfacher Feature-Engineering-Schritt habe ich deswegen für jedes Bild den **Mittelwert** und die **Standardabweichung** aller Pixel pro Kanal berechnet. Zuerst habe ich noch eine [**Standardisierung**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) durchgeführt. 

### NDVI als weiteren Kanal extrahieren 

In der Landbedeckungsklassifizierung wird oft der [Normalized Difference Vegetation Index (NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) verwendet. Dieser lässt sich einfach aus Rot- und Nahinfrarot-Kanal berechnen. Er eignet sich besonders gut, um grüne Vegetation von anderen Bodenbedeckungsklassen zu unterscheiden.

In den nachfolgenden Histogrammen, die die Verteilung der Mittelwerte für jede Kombination aus Klasse und Kanal zeigt, ist schon gut ersichtlich, dass die unterschiedlichen Klassen völlig andere Werteverteilungen haben. So haben zum Beispiel alle Klassen ausser "Grasland" und "Bäume" (und in kleinem Mass "unfruchtbares Land") einen NDVI-Mittelwert von 0.  


![Histogram nach Kanal und Klasse](img/histograms.png)


### Zwischenresultate

* Das Modell ohne NDVI ergibt pro Bild **8 Features** (2 Statistiken auf vier Kanäle). So erreichte ich auf dem erwähnten 20%-Validation-Set eine Accuracy von **98.8 Prozent**. 

* Das Modell mit zusätzlich dem NDVI hatte pro Bild **10 Features** und erreichte eine noch leicht bessere Accuracy von **99.0 Prozent**.

## Fehlerschätzung und Optimierung

Ich war nun bereits genug fortgeschritten, um mich auf diese Art von Feature Engineering und Trainingsmethode festzulegen. Nun ging es darum, die voraussichtliche Accuracy auf dem (vollständigen) Testdatensatz erwartungstreu zu schätzen und gleichzeitig die **bestmögliche Kombination aus Hyperparametern** für den Random Forest zu finden. 

### Sklearn Pipelines

Ein wesentlicher Teil dieses Projekts war die Automatisierung der Suchstrategie für die beste Hypothese. Das Package `sklearn` kann über sogenannte "Pipelines" Transformationen der Daten aneinanderhängen. Eine Transformation kann dabei Vorverarbeitung, Feature Engineering, aber auch das eigentliche Modell sein. Der Vorteil dieser Verkettung ist, dass man dadurch einheitlich Parameterräume definieren und vollautomatisch ausprobieren kann. Es können so nicht nur Parameterräume für das Modell (zum Beispiel die Anzahl Bäume im Random Forest), sondern auch für die Vorverarbeitung und das Feature Engineering ausprobiert werden. 

### Suche nach den besten Parametern mit Randomized Search und Cross Validation

Für meine Vorprozessierungsschritte habe ich keine Parameter definiert. Ob sie überhaupt angewendet werden sollen, kann jedoch auch als Parameter für die Suche nach dem besten Modell verstanden werden. Mit der Funktion `RandomizedSearchCV` habe ich **100 verschiedene Kombinationen aus Parametern** ausprobiert:

**Vorprozessierung**:
Ob der NDVI berechnet werden soll oder nicht und ob die Pixelwerte standardisiert werden sollen oder nicht.

**Modell**:
Anzahl Bäume im Random Forest, maximale Anzahl Features bei der Berechnung eines Splits in den Decision Trees, maximale Tiefe der Decision Trees, Mindestanzahl Bilder pro Blatt in den Decision Trees, ob Bootstrapping angewendet werden soll oder nicht.

Dabei wurde für jede Parameterkombination eine **3-fache Cross Validation** durchgeführt, um einen guten Schätzer für die Accuracy zu erhalten. Die Berechnung der 300 Modelle hat auf meiner Berufs-40-Kern-Workstation rund 20 Minuten gedauert.

## Endresultat

Als bestes Random-Forest-Modell wurde eines mit 1788 Bäumen, mindestens 1 Sample pro Blatt, maximaler Anzahl Features = Quadratwurzel aller Features, maximaler Tiefe von 50 und ohne Bootstrapping eruiert. Die Vorprozessierungsschritte 1) NDVI hinzufügen und 2) Werte standardisieren wurden beide beibehalten. 

Das Modell erreicht eine Accuracy von ebenfalls **99.0 Prozent** auf dem **gesamter Testdatensatz** (81'000 Bilder). Das ist nicht besser als das durch simples Ausprobieren gefundene Modell (Abschnitt Zwischenresultate). Diese Optimierung mag im aktuellen Beispiel nicht viel gebracht haben, da das Modell sowieso schon sehr gut war, aber ich kann mir vorstellen, dass sie in anderen Fällen den Mehraufwand rechtfertigt (ohne Workstation wäre die Berechnung jedoch mühsam gewesen).

### Vergleich mit anderen Ansätzen

Der Kaggle-Datensatz war zwar nicht Teil einer sogenannten "Challenge", aber es existieren ein paar Kernel, die sich mit der Problematik befassen. 

* Ein ebenso "naiver" [Ansatz](https://www.kaggle.com/kmader/deepsat-overview) (Random-Forest-Modell mit allen Spektralkanälen und ohne Feature Engineering) erreicht eine Accuracy von 95.6 Prozent auf dem Testdatensatz. Ohne Infrarotkanal sind es noch 94.2 Prozent.
* Daneben existieren noch zwei Ansätze mit Convolutional Neural Networks, die jeweils [95.1. Prozent](https://www.kaggle.com/bhumitadivarekar/simple-keras-cnn-with-95-13-accuracy) und [98.9 Prozent](https://www.kaggle.com/mrpinky/simple-cnn-accuracy-98-95-after-10-epochs) Accuracy erreichen.

## Fazit & Learnings

Mit dieser Aufgabenstellung konnte gezeigt werden, dass auch mit Machine-Learning-Methoden ausserhalb des Deep Learning Bildklassifikationen mit sehr hoher Genauigkeit durchgeführt werden können. Die Extraktion weniger, einfacher Statistiken führte nicht nur zu einer drastischen Reduktion der Dimensionalität und somit des Rechenaufwands, sondern auch zu einer Erhöhung der Accuracy um 2-3 Prozentpunkte in einem sonst schon sehr genauen Modell.

Ich vermute, dass ein Datensatz mit mehr Landbedeckungs- oder Landnutzungsklassen (anstatt "Gebäude" und "Strassen" beispielsweise auch noch "Sportplatz", "Swimming Pool" und dergleichen) eine grössere Herausforderung für einen Random Forest darstellen würde – und dann vielleicht der Schritt zu Deep Learning nötig wäre.

Meine persönlichen Learnings sind:

* Auch Methoden ausserhalb des Deep Learnings können sich für Bildklassifikation eignen.
* Domänenwissen zahlt sich aus (Stichwort NDVI).
* Ich weiss nun, wie man Vorprozessierung und Modellsuche zusammen vollautomatisch optimieren kann.